In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import time

In [2]:
engine = create_engine("postgresql://postgres@localhost:5432/arxivdb")

## Average Number of Co-authors per Paper Across Categories

$$
\begin{aligned}
R_1 &\leftarrow 
paperauthor \ast papercategory \\[8pt]
% paperauthor \bowtie_{paperauthor.arxivid = papercategory.arxivid} papercategory \\[8pt]

R_2 &\leftarrow
_{categorycode, \; arxivid}
\mathcal{F}_{count(authorid) \rightarrow author\_count}(R_1) \\[8pt]

R_3 &\leftarrow 
R_2 \ast category \\[8pt]
% R_2 \bowtie_{R_2.categorycode = category.categorycode} category \\[8pt]

R_4 &\leftarrow 
_{categoryname}
\mathcal{F}_{avg(author\_count) \rightarrow avg\_coauthors}(R_3) \\[8pt]

R_5 &\leftarrow 
\Pi_{categoryname, \; avg\_coauthors}(R_4) \\[8pt]

Result &\leftarrow
_{DESC}
\mathcal{T}_{avg\_coauthors}(R_5)
\end{aligned}
$$

In [3]:
query = """
SELECT 
    categoryname,
    AVG(author_count) AS avg_coauthors
FROM (
    SELECT 
        categorycode,
        COUNT(authorid) AS author_count
    FROM paperauthor
    NATURAL JOIN papercategory
    GROUP BY categorycode, arxivid
) sub
NATURAL JOIN category
GROUP BY categoryname
ORDER BY avg_coauthors DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 2.065 seconds.


,categoryname,avg_coauthors
0,High Energy Physics - Experiment,31.733794
1,Nuclear Experiment,18.315640
2,Instrumentation and Detectors,15.825814
3,Instrumentation and Methods for Astrophysics,13.316668
4,High Energy Astrophysical Phenomena,10.751325
5,Astrophysics of Galaxies,7.802965
6,Earth and Planetary Astrophysics,7.691134
7,Cosmology and Nongalactic Astrophysics,7.207367
8,Accelerator Physics,7.158922
9,Solar and Stellar Astrophysics,6.463935


## Average Time Between Initial Submission and Last Update

$$
\begin{aligned}
R_1 &\leftarrow 
_\text{arxivid}
\mathcal{F}_{\min(createdate) \rightarrow first\_submit, \;
\max(createdate) \rightarrow last\_update} (version) \\[8pt]

R_2 &\leftarrow 
\Pi_{arxivid, \; (last\_update - first\_submit) \rightarrow diff\_days}(R_1) \\[8pt]

Result &\leftarrow 
\mathcal{F}_{avg(diff\_days) \rightarrow avg\_days\_between}(R_2)
\end{aligned}
$$

In [4]:
query = """
SELECT 
    AVG(last_update - first_submit) AS avg_days_between
FROM (
    SELECT 
        arxivid,
        MIN(createdate) AS first_submit,
        MAX(createdate) AS last_update
    FROM version
    GROUP BY arxivid
) v;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 0.340 seconds.


,avg_days_between
0,130.72709


## Authors Who Frequently Publish Together

$$
\begin{aligned}
R_1 &\leftarrow 
\sigma_{pa1.authorid < pa2.authorid}
\big(
paperauthor(pa1) \bowtie_{pa1.arxivid = pa2.arxivid} paperauthor(pa2)
\big) \\[8pt]

R_2 &\leftarrow 
R_1 \bowtie_{pa1.authorid = a1.authorid} author(a1) 
       \bowtie_{pa2.authorid = a2.authorid} author(a2) \\[8pt]

R_3 &\leftarrow 
_{a1.authorid, \; a1.authorname, \; a2.authorid, \; a2.authorname}
\mathcal{F}_{count(arxivid) \rightarrow papers\_together}(R_2) \\[8pt]

R_4 &\leftarrow 
\sigma_{papers\_together > 3} (R_3) \\[8pt]

R_5 &\leftarrow 
\Pi_{a1.authorid \rightarrow author1\_id,\;
a1.authorname \rightarrow author1\_name,\;
a2.authorid \rightarrow author2\_id,\;
a2.authorname \rightarrow author2\_name,\;
papers\_together}(R_4) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{papers\_together}(R_5)
\end{aligned}
$$

In [5]:
query = """
SELECT 
    a1.authorid AS author1_id,
    a1.authorname AS author1_name,
    a2.authorid AS author2_id,
    a2.authorname AS author2_name,
    COUNT(*) AS papers_together
FROM paperauthor pa1
JOIN paperauthor pa2
    ON pa1.arxivid = pa2.arxivid 
    AND pa1.authorid < pa2.authorid
JOIN author a1 ON pa1.authorid = a1.authorid
JOIN author a2 ON pa2.authorid = a2.authorid
GROUP BY a1.authorid, a1.authorname, a2.authorid, a2.authorname
HAVING COUNT(*) > 3
ORDER BY papers_together DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 84.783 seconds.


,author1_id,author1_name,author2_id,author2_name,papers_together
0,558307,O. Schneider,670112,S. Eidelman,560
1,91530,P. Krokovny,558307,O. Schneider,554
2,91530,P. Krokovny,670112,S. Eidelman,554
3,32006,Y. Zhang,528798,L. Zhang,532
4,32006,Y. Zhang,540299,Y. Gao,532
5,528798,L. Zhang,540299,Y. Gao,517
6,256602,A. Bondar,670112,S. Eidelman,494
7,256602,A. Bondar,558307,O. Schneider,494
8,91530,P. Krokovny,256602,A. Bondar,492
9,558307,O. Schneider,593128,M. Shapkin,488


## Authors with the Longest Active Publishing Span

$$
\begin{aligned}
R_1 &\leftarrow 
author \ast paperauthor \ast paper \\[8pt]

R_2 &\leftarrow
_{authorid, \; authorname}
\mathcal{F}_{\min(EXTRACT(YEAR, updatedate)) \rightarrow first\_year, \;
\max(EXTRACT(YEAR, updatedate)) \rightarrow last\_year}(R_1) \\[8pt]

R_3 &\leftarrow 
\Pi_{authorid, \; authorname, \; first\_year, \; last\_year, \;
(last\_year - first\_year) \rightarrow active\_years}(R_2) \\[8pt]

Result &\leftarrow
_{DESC}
\mathcal{T}_{active\_years}(R_3)
\end{aligned}
$$

In [6]:
query = """
SELECT 
    authorid,
    authorname,
    MIN(EXTRACT(YEAR FROM updatedate)) AS first_year,
    MAX(EXTRACT(YEAR FROM updatedate)) AS last_year,
    (MAX(EXTRACT(YEAR FROM updatedate)) - MIN(EXTRACT(YEAR FROM updatedate))) AS active_years
FROM author
NATURAL JOIN paperauthor
NATURAL JOIN paper
GROUP BY authorid, authorname
ORDER BY active_years DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 5.351 seconds.


,authorid,authorname,first_year,last_year,active_years
0,10645,Enrique Ruiz Arriola,2007.0,2025.0,18.0
1,29750,Hiroya Nakao,2007.0,2025.0,18.0
2,53268,Maxim Mostovoy,2007.0,2025.0,18.0
3,37329,Brett Parker,2007.0,2025.0,18.0
4,8133,A. Eckart,2007.0,2025.0,18.0
5,42605,Sibylle Schroll,2007.0,2025.0,18.0
6,38210,Fan Yang,2007.0,2025.0,18.0
7,57548,A. Sergeev,2007.0,2025.0,18.0
8,22807,S. Tokar,2007.0,2025.0,18.0
9,35122,Vladan Pankovic,2007.0,2025.0,18.0


## Categories That Frequently Appear Together

$$
\begin{aligned}
R_1 &\leftarrow 
\sigma_{pc1.categorycode < pc2.categorycode}
\big(
papercategory(pc1) \bowtie_{pc1.arxivid = pc2.arxivid} papercategory(pc2)
\big) \\[8pt]

R_2 &\leftarrow 
R_1 \bowtie_{pc1.categorycode = c1.categorycode} category(c1)
     \bowtie_{pc2.categorycode = c2.categorycode} category(c2) \\[8pt]

R_3 &\leftarrow 
_{c1.categorycode, \; c1.categoryname, \; c2.categorycode, \; c2.categoryname}
\mathcal{F}_{count(arxivid) \rightarrow co\_occurrence}(R_2) \\[8pt]

R_4 &\leftarrow 
\Pi_{c1.categorycode \rightarrow cat1\_code,\;
c1.categoryname \rightarrow cat1\_name,\;
c2.categorycode \rightarrow cat2\_code,\;
c2.categoryname \rightarrow cat2\_name,\;
co\_occurrence}(R_3) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{co\_occurrence}(R_4)
\end{aligned}
$$

In [7]:
query = """
SELECT 
    c1.categorycode AS cat1_code,
    c1.categoryname AS cat1_name,
    c2.categorycode AS cat2_code,
    c2.categoryname AS cat2_name,
    COUNT(*) AS co_occurrence
FROM papercategory pc1
JOIN papercategory pc2 
    ON pc1.arxivid = pc2.arxivid 
   AND pc1.categorycode < pc2.categorycode
JOIN category c1 ON pc1.categorycode = c1.categorycode
JOIN category c2 ON pc2.categorycode = c2.categorycode
GROUP BY c1.categorycode, c1.categoryname, c2.categorycode, c2.categoryname
ORDER BY co_occurrence DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 0.596 seconds.


,cat1_code,cat1_name,cat2_code,cat2_name,co_occurrence
0,math-ph,Mathematical Physics,math.MP,Mathematical Physics,32863
1,cs.IT,Information Theory,math.IT,Information Theory,20913
2,gr-qc,General Relativity and Quantum Cosmology,hep-th,High Energy Physics - Theory,17922
3,hep-ph,High Energy Physics - Phenomenology,hep-th,High Energy Physics - Theory,11873
4,hep-ex,High Energy Physics - Experiment,hep-ph,High Energy Physics - Phenomenology,10973
5,cs.LG,Machine Learning,stat.ML,Machine Learning,10059
6,hep-ph,High Energy Physics - Phenomenology,nucl-th,Nuclear Theory,9988
7,math.ST,Statistics Theory,stat.TH,Statistics Theory,9075
8,astro-ph.CO,Cosmology and Nongalactic Astrophysics,gr-qc,General Relativity and Quantum Cosmology,8361
9,hep-th,High Energy Physics - Theory,math.MP,Mathematical Physics,7306


## Category-wise Yearly Publication Count

$$
\begin{aligned}
R_1 &\leftarrow 
category \ast papercategory \ast paper \\[8pt]

R_2 &\leftarrow 
\Pi_{categoryname, \; arxivid, \; EXTRACT(YEAR, updatedate) \rightarrow year}(R_1) \\[8pt]

R_3 &\leftarrow 
_{categoryname, \; year}
\mathcal{F}_{count(distinct(arxivid)) \rightarrow papers\_published}(R_2) \\[8pt]

Result &\leftarrow 
\mathcal{T}_{categoryname, year}(R_3)
\end{aligned}
$$

In [8]:
query = """
SELECT
    categoryname,
    EXTRACT(YEAR FROM updatedate) AS year,
    COUNT(DISTINCT arxivid) AS papers_published
FROM category
NATURAL JOIN papercategory
NATURAL JOIN paper
GROUP BY categoryname, year
ORDER BY categoryname, year;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(40)

Query executed in 3.097 seconds.


,categoryname,year,papers_published
0,Accelerator Physics,2007.0,15
1,Accelerator Physics,2008.0,52
2,Accelerator Physics,2009.0,112
3,Accelerator Physics,2010.0,94
4,Accelerator Physics,2011.0,129
5,Accelerator Physics,2012.0,183
6,Accelerator Physics,2013.0,233
7,Accelerator Physics,2014.0,221
8,Accelerator Physics,2015.0,561
9,Accelerator Physics,2016.0,295


## Keyword Frequency (“deep learning”) by Year

$$
\begin{aligned}
R_1 &\leftarrow 
\sigma_{\text{LOWER(title)} \; LIKE \; \text{`\%deep learning\%'}}(paper) \\[8pt]
% \; \lor \; \text{LOWER(abstract)} \; LIKE \; \text{`\%deep learning\%'}} (paper) \\[8pt]

R_2 &\leftarrow 
\Pi_{EXTRACT(YEAR, \; updatedate) \rightarrow year}(R_1) \\[8pt]

R_3 &\leftarrow 
_{year}
\mathcal{F}_{count(*) \rightarrow mentions}(R_2) \\[8pt]

Result &\leftarrow 
\mathcal{T}_{year}(R_3)
\end{aligned}
$$

In [9]:
query = """
SELECT
    EXTRACT(YEAR FROM updatedate) AS year,
    COUNT(*) AS mentions
FROM paper
WHERE LOWER(abstract) LIKE '%deep learning%'
GROUP BY year
ORDER BY year;
"""

start_time = time.time()
df = pd.read_sql(text(query), engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 2.029 seconds.


,year,mentions
0,2008.0,1
1,2010.0,1
2,2012.0,7
3,2013.0,21
4,2014.0,59
5,2015.0,169
6,2016.0,395
7,2017.0,1013
8,2018.0,1223
9,2019.0,244


## Most Common Submission Months/Days

$$
\begin{aligned}
R_1 &\leftarrow 
\Pi_{TO\_CHAR(createdate, \; \text{Month}) \rightarrow month\_name}(version) \\[8pt]

R_2 &\leftarrow 
_{month\_name}
\mathcal{F}_{count(*) \rightarrow submissions}(R_1) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{submissions}(R_2)
\end{aligned}
$$

In [10]:
query = """
SELECT 
    TO_CHAR(createdate, 'Month') AS month_name,
    COUNT(*) AS submissions
FROM version
GROUP BY month_name
ORDER BY submissions DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 0.165 seconds.


,month_name,submissions
0,May,120692
1,June,119204
2,October,119114
3,March,117481
4,November,114297
5,April,114110
6,September,112501
7,July,112092
8,December,107697
9,January,107528


$$
\begin{aligned}
R_1 &\leftarrow 
\Pi_{TO\_CHAR(createdate, \; \text{Day}) \rightarrow day\_name}(version) \\[8pt]

R_2 &\leftarrow 
_{day\_name}
\mathcal{F}_{count(*) \rightarrow submissions}(R_1) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{submissions}(R_2)
\end{aligned}
$$

In [11]:
query = """
SELECT 
    TO_CHAR(createdate, 'Day') AS day_name,
    COUNT(*) AS submissions
FROM version
GROUP BY day_name
ORDER BY submissions DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 0.137 seconds.


,day_name,submissions
0,Tuesday,249336
1,Wednesday,242467
2,Monday,242369
3,Thursday,234794
4,Friday,208005
5,Sunday,89752
6,Saturday,85394


## Number Of Distinct Categories Per Author

$$
\begin{aligned}
R_1 &\leftarrow 
author \ast paperauthor \ast papercategory \\[8pt]

R_2 &\leftarrow
_{authorid, \; authorname}
\mathcal{F}_{count(distinct(categorycode)) \rightarrow distinct\_categories}(R_1) \\[8pt]

R_3 &\leftarrow 
\Pi_{authorid, \; authorname, \; distinct\_categories}(R_2) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{distinct\_categories}(R_2) \\[8pt]
\end{aligned}
$$

In [12]:
query = """
SELECT 
    authorid,
    authorname,
    COUNT(DISTINCT categorycode) AS distinct_categories
FROM author
NATURAL JOIN paperauthor
NATURAL JOIN papercategory
GROUP BY authorid, authorname
ORDER BY distinct_categories DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 4.448 seconds.


,authorid,authorname,distinct_categories
0,228696,Wei Wang,85
1,637373,Wei Chen,84
2,316571,Wei Zhang,78
3,280453,Xin Wang,76
4,303878,Yang Liu,73
5,586466,Bo Li,72
6,548252,Lei Wang,71
7,529088,Xin Li,71
8,244966,Wei Li,71
9,162441,Wei Liu,69


## Percentage of Papers Updated More Than Once

$$
\begin{aligned}
R_1 &\leftarrow 
_{arxivid}
\mathcal{F}_{count(*) \rightarrow version\_count}(version) \\[8pt]

R_2 &\leftarrow 
\Pi_{arxivid, 
(\text{CASE: } version\_count > 1 \Rightarrow 1; \text{ else } 0) \rightarrow multi\_updated}(R_1) \\[8pt]

R_3 &\leftarrow 
\mathcal{F}_{\frac{\sum(multi\_updated)}{count(*)} \times 100 \rightarrow percent\_multi\_updated}(R_2)
\end{aligned}
$$

In [13]:
query = """
SELECT 
    ROUND(100.0 * SUM(CASE WHEN version_count > 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS percent_multi_updated
FROM (
    SELECT arxivid, COUNT(*) AS version_count
    FROM version
    GROUP BY arxivid
) sub;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 0.284 seconds.


,percent_multi_updated
0,42.48


## Proportion of Single-Author vs Multi-Author Papers

$$
\begin{aligned}
R_1 &\leftarrow 
_\text{arxivid}
\mathcal{F}_{\text{count(authorid)} \rightarrow author\_count} 
(\text{paperauthor}) \\[8pt]

R_2 &\leftarrow 
\Pi_{arxivid, \; author\_count,
(CASE: author\_count = 1 \Rightarrow \text{`Single Author'}; \; else \; \text{`Multi Author'}) 
\rightarrow paper\_type}(R_1) \\[8pt]

R_3 &\leftarrow 
_{paper\_type}
\mathcal{F}_{count(arxivid) \rightarrow num\_papers}(R_2) \\[8pt]

Result &\leftarrow 
\Pi_{paper\_type, \; num\_papers, \;
\left(\frac{num\_papers}{\sum(num\_papers)} \times 100\right) \rightarrow percentage}(R_3)
\end{aligned}
$$

In [14]:
query = """
SELECT 
    CASE 
        WHEN author_count = 1 THEN 'Single Author' 
        ELSE 'Multi Author' 
    END AS paper_type,
    COUNT(*) AS num_papers,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM (
    SELECT arxivid, COUNT(authorid) AS author_count
    FROM paperauthor
    GROUP BY arxivid
) sub
GROUP BY paper_type;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df

Query executed in 0.505 seconds.


,paper_type,num_papers,percentage
0,Single Author,177589,21.66
1,Multi Author,642447,78.34


## Submitters Who Stop Contributing After 5 Years

$$
\begin{aligned}
R_1 &\leftarrow 
author \bowtie_{author.authorid = paper.submitterid} paper \\[8pt]

R_2 &\leftarrow 
_{authorid, \; authorname}
\mathcal{F}_{\max(EXTRACT(YEAR, updatedate)) \rightarrow last\_year}(R_1) \\[8pt]

R_3 &\leftarrow 
\sigma_{last\_year < (EXTRACT(YEAR, CURRENT\_DATE) - 5)}(R_2) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{last\_year}(R_3)
\end{aligned}
$$

In [15]:
query = """
SELECT 
    a.authorid,
    a.authorname,
    MAX(EXTRACT(YEAR FROM p.updatedate)) AS last_year
FROM author a
JOIN paper p ON a.authorid = p.submitterid
GROUP BY a.authorid, a.authorname
HAVING MAX(EXTRACT(YEAR FROM p.updatedate)) < EXTRACT(YEAR FROM CURRENT_DATE) - 5
ORDER BY last_year DESC;
"""

start_time = time.time()
df = pd.read_sql(query, engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 1.425 seconds.


,authorid,authorname,last_year
0,150747,C. Hickey,2019.0
1,655846,Vera Koponen,2019.0
2,524867,Giuseppe Eugenio Bruno,2019.0
3,150534,Dominic Culver,2019.0
4,262748,Tomi Ylinen,2019.0
5,290589,Matthew Healy,2019.0
6,589040,Friederike Anna Dziemba,2019.0
7,720493,Yueyu Hu,2019.0
8,164991,Zhao-Yu Li,2019.0
9,29639,Xiaohua Li,2019.0


## Top Author by Number of Papers

$$
\begin{aligned}
R_1 &\leftarrow 
paperauthor \ast author \\[8pt]

R_2 &\leftarrow 
_{authorid, \; authorname}
\mathcal{F}_{count(arxivid) \rightarrow total\_papers\_coauthored}(R_1) \\[8pt]

Result &\leftarrow
_{DESC}
\mathcal{T}_{total\_papers\_coauthored}(R_2)
\end{aligned}
$$

In [16]:
query = """
SELECT 
    authorid,
    authorname,
    COUNT(arxivid) AS total_papers_coauthored
FROM paperauthor
NATURAL JOIN author
GROUP BY authorid, authorname
ORDER BY total_papers_coauthored DESC;
"""

start_time = time.time()
df = pd.read_sql(text(query), engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 3.867 seconds.


,authorid,authorname,total_papers_coauthored
0,32006,Y. Zhang,937
1,352272,Y. Li,705
2,528798,L. Zhang,678
3,540299,Y. Gao,617
4,265317,J. Wang,587
5,670112,S. Eidelman,578
6,558307,O. Schneider,573
7,91530,P. Krokovny,559
8,256602,A. Bondar,540
9,278694,S. Ricciardi,512


## Top Submitters by Number of Papers

$$
\begin{aligned}
R_1 &\leftarrow 
author \bowtie_{author.authorid = paper.submitterid} paper \\[8pt]

R_2 &\leftarrow 
_{authorid, authorname}
\mathcal{F}_{count(paper.arxivid) \rightarrow total\_submissions}(R_1) \\[8pt]

Result &\leftarrow 
_{DESC}
\mathcal{T}_{total\_submissions}(R_2)
\end{aligned}
$$

In [17]:
query = """
SELECT 
    a.authorid,
    a.authorname,
    COUNT(p.arxivid) AS total_submissions
FROM author a
JOIN paper p ON a.authorid = p.submitterid
GROUP BY a.authorid, a.authorname
ORDER BY total_submissions DESC;
"""

start_time = time.time()
df = pd.read_sql(text(query), engine)
end_time = time.time()

print(f"Query executed in {end_time - start_time:.3f} seconds.")
df.head(10)

Query executed in 1.397 seconds.


,authorid,authorname,total_submissions
0,85351,Delfim F. M. Torres,277
1,510708,Loet Leydesdorff,237
2,466546,Uwe Aickelin,210
3,65563,A. V,197
4,318367,William Jackson,181
5,164046,Taras Banakh,171
6,151924,Xueliang Li,169
7,582775,Bernhard Rumpe,169
8,489400,Damien Chablat,167
9,190633,Taekyun Kim,162
